In [ ]:
import pandas as pd

### Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
log = pd.read_csv('ml-latest-small/ratings.csv')
log.head()

In [ ]:
def lifetime(data):
    return data.max() - data.min()

In [ ]:
log.groupby('userId').agg({'rating' : 'count', 'timestamp' : lifetime}).query('rating > 100').timestamp.mean()

# с lambda-функцией
#log.groupby('userId').agg({'rating' : 'count', 'timestamp' : lambda x: x.max() - x.min()}).query('rating > 100').timestamp.mean()

### Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

In [ ]:
client_base[['client_id']].merge(air, how='left', on='client_id'). \
                            merge(auto, how='left', on='client_id'). \
                            merge(rzd, how='left', on='client_id').fillna(0)

In [ ]:
client_base.merge(air, how='left', on='client_id'). \
                            merge(auto, how='left', on='client_id'). \
                            merge(rzd, how='left', on='client_id').fillna(0)

### Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

#### Ответ

- Связь набора координат с конкретным визитом пользователя можно установить с помощью соотнесения времени получения координат и времени визита / совершения покупки. Кроме того, можно фиксировать информацию о расстоянии пользователя до ближайшего магазина - это увеличит точность результатов.
- При предложенном алгоритме - никак.
- Из набора координат можно получить такие признаки как, например: 
    - тип и размер населенного пункта, в котором большую часть времени находится пользователь
    - склонность пользователя к дальним перемещениям